ปฏิบัติการครั้งที่ 10 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

1. ให้เริ่มทำปฏิบัติการจาก colab notebook/ipynb ที่กำหนดให้ จากนั้น share แล้วส่ง link ใน Mango

ดาวน์โหลดข้อมูลผู้รับวินิจฉัยโรคหัวใจใน link ข้างล่างนี้  
https://donlapark.pages.dev/229351/data/framingham.csv

In [ ]:
# uploading the csv file to colab

!wget -O framingham.csv https://donlapark.pages.dev/229351/data/framingham.csv

--2024-02-22 04:14:00--  https://donlapark.pages.dev/229351/data/framingham.csv
Resolving donlapark.pages.dev (donlapark.pages.dev)... 172.66.47.56, 172.66.44.200, 2606:4700:310c::ac42:2f38, ...
Connecting to donlapark.pages.dev (donlapark.pages.dev)|172.66.47.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191803 (187K) [text/csv]
Saving to: ‘framingham.csv’

framingham.csv      100%[===================>] 187.31K  --.-KB/s    in 0.03s   

2024-02-22 04:14:00 (5.83 MB/s) - ‘framingham.csv’ saved [191803/191803]



In [ ]:
# import module ที่ต้องใช้
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('framingham.csv')

#drop rows with missing values
data = data.dropna()

print(data.info())

data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3658 entries, 0 to 4239
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   male             3658 non-null   int64  
 1   age              3658 non-null   int64  
 2   education        3658 non-null   float64
 3   currentSmoker    3658 non-null   int64  
 4   cigsPerDay       3658 non-null   float64
 5   BPMeds           3658 non-null   float64
 6   prevalentStroke  3658 non-null   int64  
 7   prevalentHyp     3658 non-null   int64  
 8   diabetes         3658 non-null   int64  
 9   totChol          3658 non-null   float64
 10  sysBP            3658 non-null   float64
 11  diaBP            3658 non-null   float64
 12  BMI              3658 non-null   float64
 13  heartRate        3658 non-null   float64
 14  glucose          3658 non-null   float64
 15  TenYearCHD       3658 non-null   int64  
dtypes: float64(9), int64(7)
memory usage: 485.8 KB
None


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


### ตัวแปรตามคือ `TenYearCHD` มีค่าเป็น 1 เมื่อผู้มาใช้บริการมีโรคหัวใจภายใน 10 ปีหลังจากการตรวจสุขภาพ

### แบ่งเป็นตัวแปรต้น `X` กับตัวแปรตาม `y`

In [ ]:
y = data['TenYearCHD']

X = data.drop(['TenYearCHD'], axis = 1)

### สร้าง Logistic regression

$$ p(y=1|x_1,\ldots,x_p) = \frac{1}{1+e^{-(a_0+a_1x_1+\ldots+a_px_p)}} $$

วิธีที่ 1: ใช้ `scikit-learn`

In [ ]:
from sklearn.linear_model import LogisticRegression

# สร้าง model object ก่อน
# ตั้ง random state ให้เป็น 0 เพื่อให้ผลลัพธ์ของทุกคนเหมือนกัน
model = LogisticRegression()

# แล้วค่อยใส่ข้อมูลโดยใช้ `fit`
model.fit(X,y)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

### Warning บอกว่ามีปํญหาในการ optimize เนื่องจากบางค่าใน `X` มีขนาดใหญ่เกินไป (ตัวอย่างเช่นตัวแปร `totChol` มีค่า `>200`)

### เป็นตัวอย่างหนึ่งที่แสดงให้เห็นถึงความสำคัญของ standardization (ปรับให้ mean ของแต่ละตัวแปรเป็น 0 และ s.d. เป็น 1)  

### สมมติให้ `X1` เป็นตัวแปรหนึ่งใน dataset  

### Standardization ทำโดยการแปลง `X1 -> (X1 - mean(X1))/(s.d.(X1))`

In [ ]:
X.mean()

male                 0.443685
age                 49.551941
education            1.980317
currentSmoker        0.489065
cigsPerDay           9.025424
BPMeds               0.030344
prevalentStroke      0.005741
prevalentHyp         0.311646
diabetes             0.027064
totChol            236.847731
sysBP              132.370558
diaBP               82.917031
BMI                 25.782802
heartRate           75.730727
glucose             81.852925
dtype: float64

In [ ]:
X.std() # หาค่าเบี่ยงเบนของแต่ละตัวแปร

male                0.496886
age                 8.562029
education           1.022656
currentSmoker       0.499949
cigsPerDay         11.921590
BPMeds              0.171557
prevalentStroke     0.075561
prevalentHyp        0.463229
diabetes            0.162292
totChol            44.097681
sysBP              22.086866
diaBP              11.974258
BMI                 4.065601
heartRate          11.981525
glucose            23.904164
dtype: float64

In [ ]:
# Standardization
X_s = (X - X.mean())/X.std()

X_s

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1.119602,-1.232411,1.974939,-0.978230,-0.757065,-0.176877,-0.075976,-0.672768,-0.166761,-0.948978,-1.193947,-1.078733,0.292010,0.356321,-0.203016
1,-0.892931,-0.414848,0.019247,-0.978230,-0.757065,-0.176877,-0.075976,-0.672768,-0.166761,0.298253,-0.514811,-0.160096,0.724911,1.608249,-0.244850
2,1.119602,-0.181259,-0.958599,1.021975,0.920563,-0.176877,-0.075976,-0.672768,-0.166761,0.184868,-0.220518,-0.243609,-0.108914,-0.060988,-0.495852
3,-0.892931,1.337073,0.997093,1.021975,1.759377,-0.176877,-0.075976,1.485991,-0.166761,-0.268670,0.798187,1.009079,0.688016,-0.895606,0.884661
4,-0.892931,-0.414848,0.997093,1.021975,1.172207,-0.176877,-0.075976,-0.672768,-0.166761,1.091946,-0.107329,0.090441,-0.659878,0.773630,0.131654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4233,1.119602,0.052331,-0.958599,1.021975,-0.673184,-0.176877,-0.075976,1.485991,-0.166761,1.726900,2.111184,0.758541,0.046044,-0.812144,0.173488
4234,1.119602,0.169126,0.997093,1.021975,2.849836,-0.176877,-0.075976,-0.672768,-0.166761,-0.676855,-0.265794,-0.243609,-1.493704,-0.895606,-0.579519
4237,-0.892931,0.285920,0.019247,-0.978230,-0.757065,-0.176877,-0.075976,-0.672768,-0.166761,0.729115,0.051136,0.006929,-1.060803,0.356321,1.051996
4238,1.119602,-1.115616,0.997093,-0.978230,-0.757065,-0.176877,-0.075976,1.485991,-0.166761,-1.175747,0.390705,1.259616,-0.044963,-0.728682,-0.412184


### ใช้ `X_s` แทน `X` แล้ว warning จะหายไป

In [ ]:
# warning จะหายไป
model.fit(X_s, y)

LogisticRegression()

In [ ]:
# ดู intercept (a_0) ที่ได้โดยใช้
print('intercept:',model.intercept_)

# ดู coefficient ที่เหลือทั้งหมด (a_1,a_2,...,a_p) โดยใช้
print('coefficients:',model.coef_)

intercept: [-1.99248411]
coefficients: [[ 0.2750739   0.54253833 -0.04900879  0.03629152  0.21257341  0.02799037
   0.0523415   0.10881813  0.00673719  0.1026998   0.33837399 -0.04841875
   0.02703884 -0.03877051  0.16991835]]


### ดูผลจากการใช้โมเดลแบ่งกลุ่ม `X_s` โดยใช้ `model.predict(X_s)`

In [ ]:
y_pred_proba = model.predict_proba(X_s)

print(y_pred_proba[:5])

[[0.96155301 0.03844699]
 [0.9509204  0.0490796 ]
 [0.84748707 0.15251293]
 [0.64600859 0.35399141]
 [0.90577064 0.09422936]]


In [ ]:
y_pred = model.predict(X_s)

print(y_pred[:40])
# probability > 0.5 => 1 เป็นโรคหัวใจใน 10 ปี
# probability < 0.5 => 0

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0]


### โมเดลแบ่งให้ข้อมูลส่วนใหญ่อยู่ในกลุ่มที่ 0 ทำให้เราไม่มั่นใจว่ามีข้อมูลที่ถูกแบ่งให้อยู่ในกลุ่มที่ 1 หรือไม่

### สามารถนับจำนวนข้อมูลที่ถูกแบ่งให้อยู่ในกลุ่มที่ 1 โดยคำนวณผลบวกของตัวเลขทั้งหมดที่อยู่ใน `y_pred`

In [ ]:
# คำนวณผลบวกของตัวเลขทั้งหมดที่อยู่ใน y_pred

print(y_pred.sum())

70


In [ ]:
y

0       0
1       0
2       0
3       1
4       0
       ..
4233    1
4234    0
4237    0
4238    0
4239    0
Name: TenYearCHD, Length: 3658, dtype: int64

### เราสามารถสร้าง array ที่ตรวจสอบว่าค่าที่ model ทำนาย (`y_pred`) ตรงกับค่าที่แท้จริง (`y`) หรือไม่โดยใช้ `y==y_pred`

| y | y_pred |       | y == y_pred |
|:------:|:------:|:-----:|:----------------:|
|    1   |    1   | $\to$ |         1        |
|    0   |    0   | $\to$ |         1        |
|    0   |    1   | $\to$ |         0        |
|    1   |    0   | $\to$ |         0        |

In [ ]:
# จะเห็นว่าโมเดลแบ่งกลุ่มให้ข้อมูลชิ้นที่ 3 ไม่ตรงกับกลุ่มที่แท้จริง
# เอาค่า y ของจริงไว้ทางด้านซ้าย ส่วนค่า y ที่ทำนายออกมาไว้ทางด้านขวา
y == y_pred

0        True
1        True
2        True
3       False
4        True
        ...  
4233    False
4234     True
4237     True
4238     True
4239     True
Name: TenYearCHD, Length: 3658, dtype: bool

In [ ]:
True + False

1

### จากตรงนี้ เราสามารถนับจำนวนข้อมูลที่โมเดลแบ่งกลุ่มได้ถูกต้องโดยใช้ `(y == y_pred).sum()` (เพราะว่าในการบวกค่า boolean นั้น `True`=0 และ `False`=1)

In [ ]:
(y == y_pred).sum()

3133

## Exercise 1:

### ให้ `y_true` เป็นค่าที่แท้จริงทั้งหมด และ `y_pred` เป็นค่าที่โมเดลทำนายทั้งหมด ค่า accuracy คำนวณได้จาก

$$\text{Accuracy} = \frac{\# (\texttt{y_true} = \texttt{y_pred})}{N}$$

### โดยที่ $N$ คือจำนวนข้อมูลทั้งหมด

### จงสร้างฟังก์ชันที่คำนวณค่า Accuracy โดยมี inputs เป็น `y_true` และ `y_pred`

### Hint: ใช้ `y_true.shape[0]` ในการนับจำนวนข้อมูลทั้งหมด

In [ ]:
def accuracy(y_true, y_pred):
  '''
  y_true: numpy array of true labels
  y_pred: numpy array of model's classifications
  Return: the accuracy of the model
  '''
  ########################
  #Fill in your code here#
  ########################
  N = y_true.shape[0]
  result = (y_true == y_pred).sum()/N
  return result

# ทดสอบฟังก์ชัน accuracy
print(accuracy(np.array([1,0,0,0]),np.array([0,1,1,1]))) #ควรจะได้ 0
print(accuracy(np.array([1,0,0,0]),np.array([1,0,0,0]))) #ควรจะได้ 1
print(accuracy(np.array([1,1,0,0]),np.array([1,0,1,1]))) #ควรจะได้ 0.25

0.0
1.0
0.25


#Exercise 1.2:

จงคำนวณ accuracy ของโมเดล logistic regression ที่ได้ข้างบน

In [ ]:
# your code here
print(accuracy(y, y_pred))

0.856478950246036


### ดูค่า Z-value และ p-value ของแต่ละตัวแปร

### ใช้ `statsmodels`

In [ ]:
from statsmodels.discrete.discrete_model import Logit

# โมเดลชื่อ logit
logit = Logit(y, X_s)

# จะเห็นว่า syntax ของ statsmodels นั้นใส่ข้อมูลก่อนแล้วค่อย fit
logit_fitted = logit.fit()

logit_fitted.summary()

Optimization terminated successfully.
         Current function value: 0.666104
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:             TenYearCHD   No. Observations:                 3658
Model:                          Logit   Df Residuals:                     3643
Method:                           MLE   Df Model:                           14
Date:                Thu, 22 Feb 2024   Pseudo R-squ.:                 -0.5613
Time:                        04:55:36   Log-Likelihood:                -2436.6
converged:                       True   LL-Null:                       -1560.6
Covariance Type:            nonrobust   LLR p-value:                     1.000
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
male                0.1187      0.037      3.187      0.001       0.046       0.192
age                 0.2497      0.040      6.286      0.000       0.172       0.328
education          -0.0234      0.035     -0.673      0.501      -0.092       0.045
currentSmoker       0.0172      0.055      0.314      0.753      -0.090       0.125
cigsPerDay          0.1090      0.056      1.935      0.053      -0.001       0.219
BPMeds              0.0403      0.038      1.065      0.287      -0.034       0.115
prevalentStroke     0.0480      0.038      1.254      0.210      -0.027       0.123
prevalentHyp        0.0492      0.049      1.013      0.311      -0.046       0.145
diabetes            0.0182      0.045      0.408      0.683      -0.069       0.106
totChol             0.0237      0.036      0.658      0.511      -0.047       0.094
sysBP               0.2383      0.068      3.530      0.000       0.106       0.371
diaBP              -0.0616      0.060     -1.034      0.301      -0.178       0.055
BMI                 0.0010      0.038      0.025      0.980      -0.074       0.076
heartRate          -0.0187      0.036     -0.526      0.599      -0.089       0.051
glucose             0.1381      0.049      2.813      0.005       0.042       0.234
===================================================================================
"""

## Exercise 2:

### จงระบุว่ามีตัวแปรไหนบ้างที่มีความสำคัญที่ระดับนัยสำคัญ 0.1

In [ ]:
#ยิ่งค่า p-value น้อยจะยิ่งมีนัยสำคัญ

คำตอบ: male, age, cigsPerDay, sysBP, glucose

### ผลลัพธ์ที่ได้จากการใช้ `predict` จะแตกต่างจาก `scikit-learn` ตรงที่ output เป็น**ค่าความน่าจะเป็น** ไม่ใช่ค่าการแบ่งกลุ่ม

In [ ]:
logit_fitted.predict(X_s)

0       0.344712
1       0.371590
2       0.520132
3       0.675797
4       0.465865
          ...   
4233    0.670071
4234    0.575457
4237    0.496277
4238    0.434234
4239    0.417771
Length: 3658, dtype: float64

## Exercise 3:

ฟังก์ชันข้างล่างนี้มีหน้าที่เปลี่ยนค่าความน่าจะเป็นที่ได้ให้เป็นการแบ่งกลุ่ม (0 หรือ 1) จงเติมส่วนที่หายไปในฟังก์ชันข้างล่างนี้ให้สมบูรณ์

In [ ]:
def classify(y_pred):
  '''
  y_pred: numpy array of probability predictions
  Return: a list of classifications (0 or 1)
  '''
  N = y_pred.shape[0] #number of observations
  results = [] #array of classifications

  for prob in y_pred:
    if prob < 0.5: #FILL HERE :
      results.append(0) #FILL HERE)
    else:
      results.append(1) #FILL HERE)

  return results


#Test
print(classify(np.array([0.2,0.4,0.8]))) #should return [0,0,1]
print(classify(np.array([0.8,0.6,0.1]))) #should return [1,1,0]

[0, 0, 1]
[1, 1, 0]


## Exercise 4:

จงแสดงการแบ่งกลุ่ม (classification) ที่ได้จากโมเดล `logit_fitted` ในพื้นที่ข้างล่างนี้


In [ ]:
#your code here
# output จะเป็น 0 กับ 1
print(classify(logit_fitted.predict(X_s)))

[0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 